In [6]:
import pandas as pd
import numpy as np

import os.path
import urllib.request 

In [8]:
#Define the data directory and file paths
#Create a directory for data if it doesn't exist
DATA_DIR = "data/"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

ATACS_SEQ_PATH = os.path.join(DATA_DIR, "ImmGenATAC18_AllOCRsInfo.csv")
RNA_SEQ_PATH = os.path.join(DATA_DIR, "mmc2.csv")
MMC1_PATH = os.path.join(DATA_DIR, "mmc1.xlsx")
REFFLAT_PATH = os.path.join(DATA_DIR, "refFlat.txt.gz")

#Check if data files are already downloaded and if not, download them
def download_with_user_agent(url, filename):
    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    with urllib.request.urlopen(req) as response, open(filename, 'wb') as out_file:
        out_file.write(response.read())

# ATAC-seq
if not os.path.isfile(ATACS_SEQ_PATH):
    download_with_user_agent("https://sharehost.hms.harvard.edu/immgen/ImmGenATAC18_AllOCRsInfo.csv", ATACS_SEQ_PATH)

# RNA-seq    
if not os.path.isfile(RNA_SEQ_PATH):
    download_with_user_agent("https://www.cell.com/cms/10.1016/j.cell.2018.12.036/attachment/4392da81-c56e-471a-b1df-0e72f03ecd77/mmc2.csv", RNA_SEQ_PATH)

if not os.path.isfile(MMC1_PATH):
    download_with_user_agent("https://www.cell.com/cms/10.1016/j.cell.2018.12.036/attachment/e5df7329-d77d-40b3-a03a-34bdbe4b402c/mmc1.xlsx", MMC1_PATH)
    
if not os.path.isfile(REFFLAT_PATH):
    download_with_user_agent("http://hgdownload.cse.ucsc.edu/goldenPath/mm10/database/refFlat.txt.gz", REFFLAT_PATH)

#Load the ATAC-seq data
atac_df = pd.read_csv(ATACS_SEQ_PATH, index_col=0)
print(atac_df.head())


                      chrom   Summit  mm10.60way.phastCons_scores  \
ImmGenATAC1219.peakID                                               
ImmGenATAC1219.peak_1  chr1  3020786                         0.00   
ImmGenATAC1219.peak_2  chr1  3087226                         0.00   
ImmGenATAC1219.peak_3  chr1  3120109                         0.07   
ImmGenATAC1219.peak_4  chr1  3121485                         0.15   
ImmGenATAC1219.peak_5  chr1  3372787                         0.03   

                       _-log10_bestPvalue  Included.in.systematic.analysis  \
ImmGenATAC1219.peakID                                                        
ImmGenATAC1219.peak_1                0.56                              NaN   
ImmGenATAC1219.peak_2                0.50                              NaN   
ImmGenATAC1219.peak_3               10.80                              1.0   
ImmGenATAC1219.peak_4                3.02                              1.0   
ImmGenATAC1219.peak_5                1.31       